# Get Started

以下命令将启动一个特定版本的容器，同时把主机当前目录映射到容器`/workspace`目录
```sh
t=flystarhe/mmdet:v2.24.1 && \
docker run --gpus all -d -p 7000:9000 --ipc=host --name test -v "$(pwd)":/workspace ${t} notebook
```

这时你有两种方式来开始训练或验证任务：

- Jupyter - http://hostname:7000/?token=hi
- Bash - `docker exec -it test bash`

本教程只针对Jupyter方式。目标为提高工作的效率和规范性，实现所有参与者聚焦[Perception Ecosystem](#)第二节的模式快速开展工作，而不是每位同时都需要重复的去和环境作斗争。

## Prepare Config
如果你想导出一份版本库里原始的
```python
from mmcv import Config

MMDET_HOME = "/opt/src/mmdetection"
CONFIG_FILE = "configs/centernet/centernet_resnet18_140e_coco.py"
cfg = Config.fromfile(f"{MMDET_HOME}/{CONFIG_FILE}")
cfg.dump(f"/workspace/centernet_resnet18.py")
print(f"Config:\n{cfg.pretty_text}")
```

## Prepare Data

In [1]:
%%bash
rm -rf coco2017_cat_dog*
wget -q https://github.com/flystarhe/containers/releases/download/v0.2.0/coco2017_cat_dog.tar
tar -xf coco2017_cat_dog.tar
ls -Alp .

total 1724
drwxr-xr-x 2 root root    4096 May 31 14:45 .ipynb_checkpoints/
-rw-r--r-- 1 root root    8437 Jun  1 08:42 centernet_resnet18.py
drwxr-xr-x 4 root root    4096 May 17 21:19 coco2017_cat_dog/
-rw-r--r-- 1 root root 1669120 May 31 08:56 coco2017_cat_dog.tar
-rw-r--r-- 1 root root   65857 Jun  2 08:53 get_started.ipynb
drwxr-xr-x 4 root root    4096 May 31 18:35 tmp/


```log
coco2017_cat_dog
├── train
│   ├── data
│   │   ├── 000000001319.jpg
│   │   ├── 000000011635.jpg
│   │   ├── 000000012764.jpg
│   │   ├── 000000013636.jpg
│   │   ├── 000000016759.jpg
│   │   └── 000000018155.jpg
│   └── labels.json
└── validation
    ├── data
    │   ├── 000000022892.jpg
    │   ├── 000000071226.jpg
    │   ├── 000000169076.jpg
    │   ├── 000000179392.jpg
    │   ├── 000000189806.jpg
    │   └── 000000219578.jpg
    └── labels.json
```

## Train

### Training on a single GPU

In [2]:
MMDET_HOME = "/opt/src/mmdetection"
CONFIG_FILE = f"{MMDET_HOME}/configs/centernet/centernet_resnet18_140e_coco.py"
EXPERIMENT_PATH = "/workspace/tmp/runs/coco2017_cat_dog_centernet_resnet18"
DATA_ROOT = "/workspace/coco2017_cat_dog"
NUM_CLASSES = "2"
CLASSES = "cat,dog"

ARGS = [f"{CONFIG_FILE}",
        f"--work-dir {EXPERIMENT_PATH}",
        f"--seed 41",
        f"--launcher none",
        f"--cfg-options",
        f"'model.bbox_head.num_classes={NUM_CLASSES}'",
        f"'data.samples_per_gpu=2'",
        f"'data.workers_per_gpu=2'",
        f"'data.train.dataset.classes={CLASSES}'",
        f"'data.train.dataset.ann_file={DATA_ROOT}/train/labels.json'",
        f"'data.train.dataset.img_prefix={DATA_ROOT}/train/data/'",
        f"'data.val.classes={CLASSES}'",
        f"'data.val.ann_file={DATA_ROOT}/validation/labels.json'",
        f"'data.val.img_prefix={DATA_ROOT}/validation/data/'",
        f"'data.test.classes={CLASSES}'",
        f"'data.test.ann_file={DATA_ROOT}/validation/labels.json'",
        f"'data.test.img_prefix={DATA_ROOT}/validation/data/'",
        "'log_config.interval=8'",  # 50, 100 or 200
        "'runner.max_epochs=3'"]  # 12, 24 or 30
ARGS = " ".join(ARGS)
print(f"{ARGS=}")

!cd {MMDET_HOME} && python tools/train.py {ARGS}

ARGS="/opt/src/mmdetection/configs/centernet/centernet_resnet18_140e_coco.py --work-dir /workspace/tmp/runs/coco2017_cat_dog_centernet_resnet18 --seed 41 --launcher none --cfg-options 'model.bbox_head.num_classes=2' 'data.samples_per_gpu=2' 'data.workers_per_gpu=2' 'data.train.dataset.classes=cat,dog' 'data.train.dataset.ann_file=/workspace/coco2017_cat_dog/train/labels.json' 'data.train.dataset.img_prefix=/workspace/coco2017_cat_dog/train/data/' 'data.val.classes=cat,dog' 'data.val.ann_file=/workspace/coco2017_cat_dog/validation/labels.json' 'data.val.img_prefix=/workspace/coco2017_cat_dog/validation/data/' 'data.test.classes=cat,dog' 'data.test.ann_file=/workspace/coco2017_cat_dog/validation/labels.json' 'data.test.img_prefix=/workspace/coco2017_cat_dog/validation/data/' 'log_config.interval=8' 'runner.max_epochs=3'"
/opt/src/mmdetection/mmdet/utils/setup_env.py:38: UserWarning: Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system bei

### Training on multiple GPUs

In [3]:
CFG_NUM_GPUS = 1
MMDET_HOME = "/opt/src/mmdetection"
CONFIG_FILE = f"{MMDET_HOME}/configs/centernet/centernet_resnet18_140e_coco.py"
EXPERIMENT_PATH = "/workspace/tmp/runs/coco2017_cat_dog_centernet_resnet18"
DATA_ROOT = "/workspace/coco2017_cat_dog"
NUM_CLASSES = "2"
CLASSES = "cat,dog"

ARGS = [f"{CONFIG_FILE}",
        f"--work-dir {EXPERIMENT_PATH}",
        f"--seed 41",
        f"--launcher pytorch",  # for distributed
        f"--cfg-options",
        f"'model.bbox_head.num_classes={NUM_CLASSES}'",
        f"'data.samples_per_gpu=2'",
        f"'data.workers_per_gpu=2'",
        f"'data.train.dataset.classes={CLASSES}'",
        f"'data.train.dataset.ann_file={DATA_ROOT}/train/labels.json'",
        f"'data.train.dataset.img_prefix={DATA_ROOT}/train/data/'",
        f"'data.val.classes={CLASSES}'",
        f"'data.val.ann_file={DATA_ROOT}/validation/labels.json'",
        f"'data.val.img_prefix={DATA_ROOT}/validation/data/'",
        f"'data.test.classes={CLASSES}'",
        f"'data.test.ann_file={DATA_ROOT}/validation/labels.json'",
        f"'data.test.img_prefix={DATA_ROOT}/validation/data/'",
        "'log_config.interval=8'",  # 50, 100 or 200
        "'runner.max_epochs=3'"]  # 12, 24 or 30
ARGS = " ".join(ARGS)
print(f"{ARGS=}")

!cd {MMDET_HOME} && python -m torch.distributed.run --nproc_per_node {CFG_NUM_GPUS} tools/train.py {ARGS}

ARGS="/opt/src/mmdetection/configs/centernet/centernet_resnet18_140e_coco.py --work-dir /workspace/tmp/runs/coco2017_cat_dog_centernet_resnet18 --seed 41 --launcher pytorch --cfg-options 'model.bbox_head.num_classes=2' 'data.samples_per_gpu=2' 'data.workers_per_gpu=2' 'data.train.dataset.classes=cat,dog' 'data.train.dataset.ann_file=/workspace/coco2017_cat_dog/train/labels.json' 'data.train.dataset.img_prefix=/workspace/coco2017_cat_dog/train/data/' 'data.val.classes=cat,dog' 'data.val.ann_file=/workspace/coco2017_cat_dog/validation/labels.json' 'data.val.img_prefix=/workspace/coco2017_cat_dog/validation/data/' 'data.test.classes=cat,dog' 'data.test.ann_file=/workspace/coco2017_cat_dog/validation/labels.json' 'data.test.img_prefix=/workspace/coco2017_cat_dog/validation/data/' 'log_config.interval=8' 'runner.max_epochs=3'"
/opt/src/mmdetection/mmdet/utils/setup_env.py:38: UserWarning: Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system 